<a href="https://colab.research.google.com/github/lazyshyman/practice_place/blob/main/Ai_individual_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [9]:
# Define device - this allows you to use a GPU if available, otherwise it defaults to the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Step 1: Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [11]:
# Step 2: Define the Neural Network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Fully connected layers
        self.fc1 = nn.Linear(28 * 28, 128)  # input layer (28*28 pixels), 128 hidden units
        self.fc2 = nn.Linear(128, 64)       # hidden layer 1 with 64 hidden units
        self.fc3 = nn.Linear(64, 10)        # output layer (10 digits, 0-9)

    def forward(self, x):
        # Flatten the image from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        x = x.view(-1, 28*28)
        # Pass through the fully connected layers with ReLU activation functions
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # Output layer with no activation (we'll use CrossEntropyLoss which applies softmax)
        x = self.fc3(x)
        return x

In [12]:
# Step 3: Instantiate the model, define the loss function and optimizer
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()  # Loss function for classification problems
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001


In [13]:
# Step 4: Train the model
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to the device (GPU/CPU)

        # Zero the gradients before backward pass
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.2723
Epoch [2/5], Loss: 0.1159
Epoch [3/5], Loss: 0.0815
Epoch [4/5], Loss: 0.0635
Epoch [5/5], Loss: 0.0501


In [14]:
# Step 5: Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the highest logit (class prediction)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 97.47%
